In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import math

import optimagic as om
import scipy

import warnings
warnings.filterwarnings('ignore')

# Numerical Optimization

In [ ]:
def square(params):
    x = params
    return (x-4)**2

def sq_gradient(params):
    x = params
    return 2*(x-4)

def sphere(params):
    return params @ params

In [ ]:
bounds = om.Bounds(lower = 5, upper = 10)
lbfgsb_res = om.minimize(
    fun=square,
    params=9,
    algorithm="scipy_lbfgsb",
    jac = sq_gradient,
    bounds = bounds,
    logging = 'my_log.db',
    log_options = {'if_database_exists':"replace"},
)

lbfgsb_res.params

In [ ]:
lbfgsb_res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    constraints = om.FixedConstraint(selector = lambda params : params[4]),
)

lbfgsb_res.params.round(3)

In [ ]:
fig = om.params_plot(lbfgsb_res)
fig.show()

In [ ]:
lbfgsb_res = om.minimize(
    fun=sphere,
    params=np.array([0.2,0.8]),
    algorithm="scipy_lbfgsb",
    constraints = om.ProbabilityConstraint(selector = lambda params : params),
)

lbfgsb_res.params.round(3)

In [ ]:
nm_res = om.minimize(
    fun=square,
    params=9,
    algorithm="scipy_neldermead",
    bounds = bounds,
)

lbfgsb_res.params

In [ ]:
@om.mark.least_squares
def ls_sphere(params):
    return params

In [ ]:
res = om.minimize(
    fun = ls_sphere,
    params =  np.arange(5),
    algorithm  = 'scipy_neldermead',
)
res.params.round(3)

In [ ]:
results = {"lbfgsb": lbfgsb_res, "nelder_mead": nm_res}
om.criterion_plot(results, max_evaluations = 30)

In [ ]:
fig = om.params_plot(
    lbfgsb_res,
    max_evaluations=300,
    # optionally select a subset of parameters to plot
    selector=lambda params: params,
)
fig.show(renderer="png")

reader = om.OptimizeLogReader("my_log.db")
reader.read_history()['params']

# Numerical Differerntiation

In [ ]:
def sphere(params):
    return params @ params

In [ ]:
params = np.arange(5)
fd = om.first_derivative(
    func = sphere,
    params = params,
    method = 'backward',
    bounds = om.Bounds(lower = params, upper = params + 5)
)

In [ ]:
fd.derivative

In [ ]:
sd = om.second_derivative(
    func = sphere,
    params = np.arange(5),
    n_cores = 6,
    method = 'forward',
)

In [ ]:
sd.derivative.round(3)

##  Objective functions

In [ ]:
import pandas as pd

def sphere(params):
    return (params["value"]**2).sum()

@om.mark.least_squares
def ls_spheres(params):
    return params['value']
params = pd.DataFrame(
    data = {"value" : [1,2] , "bounds" : [0,1]}
)
om.minimize(
    fun = ls_sphere,
    params = params,
    algorithm = 'scipy_lbfgsb',
    bounds = om.Bounds(lower = params['bounds'])
).params.round(3)

In [ ]:
def shifted_sphere(x, offset):
    return (x-offset) @ (x - offset)

om.minimize(
    fun = shifted_sphere,
    params = np.arange(5),
    algorithm = 'scipy_lbfgsb',
    fun_kwargs = {'offset': 2}
).params.round(2)

In [ ]:
x = np.arange(20)
x = np.append(x,0)
x = np.delete(x, [0])
x

In [ ]:
om.algos.GradientBased.Scalar.Available

In [ ]:
x0 = np.array([2,3,4])

In [ ]:
x1 = np.ones_like(x0)
x1

# Logging

In [ ]:
log_options = om.SQLiteLogOptions(
    "my_log.db",
    fast_logging=True,
    if_database_exists=om.ExistenceStrategy.REPLACE,
)

res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    logging=log_options,
)

In [ ]:
reader = res.logger
reader.read_start_params()

In [ ]:
reader.read_history()['fun']

In [ ]:
import sys
sys.path.insert(0, "/home/gaurav/Documents/")
import os
import optimagic as om
fig = om.criterion_plot(res)
fig.show()



In [ ]:
print(om.__file__)


In [ ]:
fig = om.params_plot("my_log.db")
fig.show()

In [ ]:
params = {"alpha": 0, "beta": 0, "gamma": 0, "delta": 0}
for key in params:
    print(key)

In [ ]:
res = om.minimize(
    fun= lambda x : x @ x,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    logging= om.SQLiteLogOptions(
    "my_log.db", if_database_exists=om.ExistenceStrategy.REPLACE,),
)
fig = om.criterion_plot("my_log.db")
fig.show()

In [ ]:
import optimagic as om
import numpy as np
log_options = om.SQLiteLogOptions("my_log.db", if_database_exists=om.ExistenceStrategy.REPLACE,)
res = om.minimize(fun=lambda x: x @ x, params=np.arange(5), algorithm="scipy_lbfgsb", logging=log_options,)




In [ ]:
om.minimize(
    fun= lambda x : x@x,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    logging=om.SQLiteLogOptions(
    "my_log.db", if_database_exists=om.ExistenceStrategy.REPLACE
),)
fig = om.criterion_plot("my_log.db")
fig.show()

In [ ]:
from pathlib import Path
path = Path("test_log.db")

om.minimize(
    fun= lambda x : x@x,
    params=np.arange(10),
    algorithm="scipy_lbfgsb",
    logging=om.SQLiteLogOptions(
    path, if_database_exists=om.ExistenceStrategy.REPLACE
),)
fig = om.criterion_plot(path)
fig.show()

# Benchmark test bug scipy

In [2]:
import scipy
import pickle
import optimagic as om

In [ ]:
# print(scipy.__version__)
# problems = om.get_benchmark_problems("more_wild")
# optimizers = {
#     "scipy_lbfgsb"
# }
# results_scipy_1_15_2 = om.run_benchmark(problems,
#                        optimizers)
# with open("results_scipy_1_15_2", "wb") as f:
#     pickle.dump(results_scipy_1_15_2, f)

In [ ]:
# print(scipy.__version__)
# problems = om.get_benchmark_problems("more_wild")
# optimizers = {
#     "scipy_lbfgsb"
# }
# results_scipy_1_14_0 = om.run_benchmark(problems,
#                        optimizers)
# with open("results_scipy_1_14_0", "wb") as f:
#     pickle.dump(results_scipy_1_14_0, f)


1.14.0


In [14]:
with open("scipy_old/results_scipy_1_14_0", "rb") as f:
    results_scipy_1_14_0 = pickle.load(f)
with open("scipy_old/results_scipy_1_15_2", "rb") as f:
    results_scipy_1_15_2 = pickle.load(f)
problems = om.get_benchmark_problems("more_wild")
updated_results_scipy_1_14_0 = {}
updated_results_scipy_1_15_2 = {}
for key, value in results_scipy_1_14_0.items():
    (a,b) = key
    updated_results_scipy_1_14_0[(a,'scipy_1-14_0_lbfgsb')] = results_scipy_1_14_0[key]
for key, value in results_scipy_1_15_2.items():
    (a,b) = key
    updated_results_scipy_1_15_2[(a,'scipy_1-15_2_lbfgsb')] = results_scipy_1_15_2[key]

print(updated_results_scipy_1_15_2.keys())
results = updated_results_scipy_1_14_0 | updated_results_scipy_1_15_2
print(results.keys())

dict_keys([('linear_full_rank_good_start', 'scipy_1-15_2_lbfgsb'), ('linear_full_rank_bad_start', 'scipy_1-15_2_lbfgsb'), ('linear_rank_one_good_start', 'scipy_1-15_2_lbfgsb'), ('linear_rank_one_bad_start', 'scipy_1-15_2_lbfgsb'), ('linear_rank_one_zero_columns_rows_good_start', 'scipy_1-15_2_lbfgsb'), ('linear_rank_one_zero_columns_rows_bad_start', 'scipy_1-15_2_lbfgsb'), ('rosenbrock_good_start', 'scipy_1-15_2_lbfgsb'), ('rosenbrock_bad_start', 'scipy_1-15_2_lbfgsb'), ('helical_valley_good_start', 'scipy_1-15_2_lbfgsb'), ('helical_valley_bad_start', 'scipy_1-15_2_lbfgsb'), ('powell_singular_good_start', 'scipy_1-15_2_lbfgsb'), ('powell_singular_bad_start', 'scipy_1-15_2_lbfgsb'), ('freudenstein_roth_good_start', 'scipy_1-15_2_lbfgsb'), ('freudenstein_roth_bad_start', 'scipy_1-15_2_lbfgsb'), ('bard_good_start', 'scipy_1-15_2_lbfgsb'), ('bard_bad_start', 'scipy_1-15_2_lbfgsb'), ('kowalik_osborne', 'scipy_1-15_2_lbfgsb'), ('meyer', 'scipy_1-15_2_lbfgsb'), ('watson_6_good_start', 'scipy_

In [25]:
fig = om.profile_plot(problems, results, y_precision=0.00001)
fig.show()
# fig.write_image("scipy_old/yp=0.000001.png", scale=2)

In [26]:
fig = om.convergence_plot(
    problems=problems,
    results=results,
    n_cols=2,
    )

# fig.write_image("scipy_old/convplot.png", scale=2)
fig.show()

In [19]:
df = om.convergence_report(
    problems=problems,
    results=results_scipy_1_15_2,
    stopping_criterion="y",
    x_precision=1e-4,
    y_precision=1e-4,
)
df

,scipy_lbfgsb,dimensionality
problem,,
bard_bad_start,success,3
bard_good_start,success,3
bdqrtic_10,success,10
bdqrtic_11,success,11
bdqrtic_12,success,12
bdqrtic_8,success,8
box_3d,success,3
brown_almost_linear,success,10
brown_almost_linear_medium,success,100


# Make fun optional

In [ ]:
def sphere(x):
    return x @ x
def grad_sphere(x):
    return 2*x
def sphere_and_grad(x):
    return sphere(x), grad_sphere(x)

In [ ]:
res = om.minimize(
    # fun = sphere,
    params = np.arange(5),
    # jac = grad_sphere,
    fun_and_jac = lambda x: (x@x, 2*x),
    algorithm='scipy_lbfgsb',   
    
)
res.params

array([ 0.00000000e+00, -5.55111512e-16, -1.11022302e-15, -2.22044605e-15,
       -2.22044605e-15])

In [ ]:
expected = om.minimize(
        fun=lambda x: x @ x,
        jac = lambda x: 2*x,
        params=np.arange(5),
        algorithm="scipy_lbfgsb",
    )
res = om.minimize(
        fun_and_jac=lambda x: (x @ x, 2 * x),
        params=np.arange(5),
        algorithm="scipy_lbfgsb",
    )
np.all(res.x != expected.x)


np.False_

In [11]:
@om.mark.least_squares
def ls_sphere(params):
    return params


@om.mark.least_squares
def ls_sphere_jac(params):
    return np.eye(len(params))

def sphere_gradient(x):
    return 2*x

res = om.minimize(
    fun=ls_sphere,
    params=np.arange(3),
    algorithm="scipy_ls_lm",
    jac=ls_sphere_jac,
)
res.params.round(5)
results = {}
for algorithm in ["scipy_ls_lm"]:
    results[algorithm] = om.minimize(
        params=np.arange(5),
        algorithm=algorithm,
        fun_and_jac=lambda x:(x@x, 2*x),
    )

fig = om.criterion_plot(results)
fig.show(renderer="png")

InvalidFunctionError: Least-squares solvers can only be used with least-squares problems.